# Evaluation and calibration with uncertain ground truth

This tutorial goes through Google DeepMind's [Uncertain Ground Truth (UGT)](https://github.com/google-deepmind/uncertain_ground_truth) framework for evaluating and calibrating machine learning models under uncertain ground truth. The tutorial is based on the paper [Conformal prediction under ambiguous ground truth](https://openreview.net/forum?id=CAd6V2qXxc) [1] and the case study [Evaluating AI systems under uncertain ground truth: a case study in dermatology](https://arxiv.org/abs/2307.02191) [2].

The tutorial will look to explain how the approach can be used to other experimental settings, and how it can be used to evaluate and calibrate machine learning models.